Imports

In [ ]:
import imaplib
import ssl
from icecream import ic
import email
import os
import builtins
import email
from email.header import decode_header
from datetime import datetime, timedelta
import io
import psycopg2
from configparser import ConfigParser
import os
import requests
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import Column, Integer, String, DateTime, ARRAY
from sqlalchemy.orm import sessionmaker, declarative_base
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

# Create the SQLAlchemy base class
Base = declarative_base()

Databae Models

In [ ]:
# Define the Email and Attachment models using SQLAlchemy ORM
class Email(Base):
    __tablename__ = 'email_orders'
    email_order_id = Column(Integer, primary_key=True)
    email_id = Column(String)
    email_from = Column(String)
    email_time = Column(DateTime)
    email_subject = Column(String)
    attachment_filename = Column(String)
    attachment_type = Column(String)
    attachment_link = Column(String)
    attachment_save_location = Column(String)
    status = Column(String(20))
    status_change_date = Column(DateTime)

    def __init__(self, 
                 email_id=None, 
                 email_from=None, 
                 email_time=None, 
                 email_subject=None, 
                 attachment_filename=None,  
                 attachment_type=None, 
                 attachment_link=None, 
                 email_order_id=None, 
                 attachment_save_location=None, 
                 status=None, 
                 status_change_date=None
                ):
        self.email_id = email_id
        self.email_order_id = email_order_id
        self.email_from = email_from
        self.email_time = email_time
        self.email_subject = email_subject
        self.attachment_filename = attachment_filename
        self.attachment_type = attachment_type
        self.attachment_link = attachment_link
        self.attachment_save_location = attachment_save_location
        self.status = status
        self.status_change_date = status_change_date
    
class Email_Metadata(db.Model):
    __tablename__ = 'email_metadata'
    
    email_metadata_id = Column(Integer, primary_key=True)
    email_id = Column(String)
    recipients = Column(ARRAY(String))
    cc = Column(ARRAY(String))
    bcc = Column(ARRAY(String))
    flags = Column(String)
    importance = Column(String)
    categories = Column(String)
    body = Column(String)
    thread_id = Column(String)
    headers = Column(String)
    folder = Column(String)
    email_size = Column(Integer)
    status = Column(String(20))
    email_order_id = Column(Integer)
    status_change_date = Column(DateTime)

    def __init__(self, 
                 email_metadata_id=None,
                 email_id=None, 
                 recipients=None, 
                 cc=None, 
                 bcc=None, 
                 flags=None, 
                 importance=None, 
                 categories=None, 
                 body=None, 
                 thread_id=None, 
                 headers=None, 
                 folder=None,
                 email_size=None, 
                 status=None, 
                 status_change_date=None
                ):
        self.email_id = email_id
        self.recipients = recipients
        self.cc = cc
        self.bcc = bcc
        self.flags = flags
        self.importance = importance
        self.categories = categories
        self.body = body
        self.thread_id = thread_id
        self.headers = headers
        self.folder = folder
        self.email_size = email_size
        self.status = status
        self.status_change_date = status_change_date

Get Context for IMAP Server Connection

In [ ]:
def getImapConfig():
    returnValue = None
    try:
        context = ssl.create_default_context()
        context.check_hostname = False
        context.verify_mode = ssl.CERT_NONE
        context.ssl_version = ssl.PROTOCOL_TLSv1_2
        
        returnValue = context
    except Exception as e:
        print(f"getImapConfig > Error : {e}")
        
    return returnValue

Get Email Connection Credentials

In [ ]:
def getEmailCredentials():
    returnValue = None
    try:
        config_filepath = os.path.join("config.ini")
        # check if the config file exists
        exists = os.path.exists(config_filepath)
        config = None
        if exists:
            print(f"getEmailCredentials > config.ini file found")
            config = ConfigParser()
            config.read(config_filepath)
            returnValue = config.get('EMAIL_CREDENTIALS','USERNAME'), config.get('EMAIL_CREDENTIALS','PASSWORD')
        else:
            print(f"getEmailCredentials > config.ini file not found")
    except Exception as e:
        print(f"getEmailCredentials > Error : {e}")
        
    return returnValue

Get Database Connection Credentials

In [ ]:
def getDatabaseConfig():
    returnValue = None
    try:
        config_filepath = os.path.join("config.ini")
        # check if the config file exists
        exists = os.path.exists(config_filepath)
        config = None
        if exists:
            config = ConfigParser()
            config.read(config_filepath)
            returnValue = config
        else:
            print(f"getDatabaseCredentials > config.ini file not found")
    except Exception as e:
        print(f"getDatabaseCredentials > Error : {e}")
        
    return returnValue

Get IMap server object which Connects To Imap Server with 993 port and SSL Context

In [ ]:
def get_imap_server():
    returnValue = None
    try:
        # Connect to the Gmail IMAP server
        imap_server = imaplib.IMAP4_SSL('imap.gmail.com', port=993, ssl_context=getImapConfig())
        returnValue = imap_server
    except Exception as e:
        print(f"get_imap_server > Error : {e}")
        
    return returnValue

Method to save Email Attachments

In [ ]:
def saveEmailAttachment(part):
    returnValue = False
    try:
        # Get the filename and extension
        filename = part.get_filename()
        
        # Save the attachment to a separate directory
        save_path = os.path.join('EmailAttachment')
        
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        
        if filename:
            
            attachment_path = os.path.join(save_path, filename)
            with open(attachment_path, 'wb') as attachment_file:
                attachment_file.write(part.get_payload(decode=True))
                
            returnValue = attachment_path
    
    except Exception as e:
        print(f"saveEmailAttachment > Error : {e}")
        
    return returnValue

Method to save Email Attachment from Links

In [ ]:
def saveLinkData(part):
    returnValue = False
    try:
    
        # Handle other attachment types (e.g., PDF, Word, Excel, etc.)
        if part.get_filename() is None:
            # Handle link attachments
            link_url = part.get_payload()
            # Download the data from the link
            response = requests.get(link_url)
            # Extract the filename from the link URL
            filename = link_url.split('/')[-1]
            
            # Save the attachment to a separate directory
            save_path = os.path.join('EmailAttachment')
            
            if not os.path.exists(save_path):
                os.makedirs(save_path)
                
            if filename:
                attachment_path = os.path.join(save_path, filename)
                with open(attachment_path, 'wb') as attachment_file:
                    attachment_file.write(part.get_payload(decode=True))
                
    except Exception as e:
        print(f"saveLinkData > Error : {e}")
        
    return returnValue

Get Serch Paramseters

In [ ]:
# Get search parameters (e.g., time range, email addresses)
def get_serach_string(email_from):
    returnValue = None
    try:
        # Calculate the start time as 11 am of the last 7th day
        start_time = datetime.now() - timedelta(days=7)
        start_time = start_time.replace(hour=11, minute=0, second=0, microsecond=0)

        # Define the end time as the current time
        end_time = datetime.now()

        # Format the time range for the search query
        start_time_str = start_time.strftime('%d-%b-%Y') # %H:%M:%S')
        end_time_str = end_time.strftime('%d-%b-%Y %H:%M:%S')

        formatted_date_range = f'SENTSINCE "{start_time_str}"'

        search_criteria = f'UNSEEN FROM "{email_from}" {formatted_date_range}'
        returnValue = search_criteria
    except Exception as e:
        print(f"get_search_params > Error > {e}")
        
    return returnValue

Get Email Data

In [ ]:
def get_email_data(imap_server:imaplib.IMAP4_SSL, message_id):
    returnValue = None
    try:
        print(f"get_email_data > {message_id}")
        _, data = imap_server.fetch(message_id, '(RFC822)')
        raw_email = data[0][1]
        email_message = email.message_from_bytes(raw_email)
        returnValue =  raw_email, email_message
    except Exception as e:
        print(f"get_email_data > Error : {e}")

    return returnValue

Extract Email Metadata

In [ ]:
def extract_email_metadata(email_message):
    returnValue = None
    try:
        email_id = email_message.get('Message-ID')
        email_from = email_message.get('From')
        email_time = email_message.get('Date')
        email_subject = email_message.get('Subject')
        
        returnValue = {
            'email_id': email_id,
            'email_from': email_from,
            'email_time': email_time,
            'email_subject': email_subject
        }
        
    except Exception as e:
        print(f"extract_email_metadata > Error : {e}")
    
    return returnValue


In [ ]:
def extract_attachments(email_message):
    returnValue = None
    try:
        attachments = []
        
        for part in email_message.walk():
            if part.get_content_maintype() == 'multipart':
                continue
            if part.get('Content-Disposition') is None:
                continue
            
            filename = part.get_filename()
            content_type = part.get_content_type()
            attachment_data = part.get_payload(decode=True)
            
            attachment = {
                'filename': filename,
                'content_type': content_type,
                'data': attachment_data,
                'part': part
            }
            
            attachments.append(attachment)
    
        returnValue = attachments
    
    except Exception as e:
            print(f"extract_attachments > Error : {e}")
        
    return returnValue


Save Email Data

In [ ]:
def save_email_data(session, email_data, attachment_data, additional_data):
    try:
        
        print(f"Saving data for {email_data['email_id']}")
        
        email_order_id = session.query(func.wa.getseq()).one()._data[0]
        
        # Create an Email instance
        email = Email(
            email_order_id = email_order_id,
            email_id=email_data['email_id'],
            email_from=email_data['email_from'],
            email_time=email_data['email_time'],
            email_subject=email_data['email_subject'],
            status = 'PEND'
        )
           
        # Create Attachment instances
        attachments = []
        for attachment in attachment_data:
            print(f"Saving attachment data for {email_data['email_id']}")
            
            email.attachment_save_location = saveEmailAttachment(attachment['part'])
            email.attachment_filename= attachment['filename'],
            email.attachment_type=attachment['content_type']
            
        # Add the email to the session
        
        email.status_change_date = datetime.now()
        
        session.add(email)
        
        # Commit the changes to the database
        session.commit()
        
        email_metadata_id = session.query(func.wa.getseq()).one()._data[0]
        
        print(f"Saving email metadata for {email_data['email_id']}")
        
        # Create an EmailsAdditional instance
        emails_additional = Email_Metadata(
            email_metadata_id = email_metadata_id,
            email_id=email_data['email_id'],
            recipients=additional_data['recipients'],
            cc=additional_data['cc'],
            bcc=additional_data['bcc'],
            flags=additional_data['flags'],
            importance=additional_data['importance'],
            categories=additional_data['categories'],
            body=additional_data['body'],
            thread_id=additional_data['thread_id'],
            # headers=additional_data['headers'],
            # folder=additional_data['folder'],
            email_size=additional_data['email_size'],
            status = 'PEND',
            status_change_date = datetime.now()
        )
        
        emails_additional.email_order_id = email_order_id
        
        # Add the EmailsAdditional to the session
        session.add(emails_additional)
        
        # Commit the changes to the database
        session.commit()
    except SQLAlchemyError as e:
        print(f"An error occurred while saving email data: {str(e)}")
        session.rollback()
        raise


Get Additional Data from Email (Email Metadata)

In [ ]:
def get_email_metadata(email_id, imap_server):
    returnValue = None
    try:
        # _, email_data = imap_server.fetch(email_id, '(RFC822 X-GM-MSGID X-GM-THRID X-GM-LABELS X-GM-IMPORTANCE FLAGS)')

        _, email_data = imap_server.fetch(email_id, '(RFC822)')

        raw_email = email_data[0][1]
        email_message = email.message_from_bytes(raw_email)
    
        ic(email_message.__dict__)
    
        # msg_id = email_message.get('X-GM-MSGID', None)
        # thread_id = email_message.get('X-GM-THRID', None)
        # labels = email_message.get('X-GM-LABELS', '')
        # importance = email_message.get('X-GM-IMPORTANCE', '')
        # flags = email_message.get('FLAGS', '')
        
        msg_id = email_message.get('Message-ID', None)
        thread_id = email_message.get('References', None)
        labels = email_message.get('X-Gmail-Labels', '')
        importance = email_message.get('Importance', '')
        flags = email_message.get('X-Gmail-Message-State', '')
        email_time = email_message.get('Date')

        # Get the recipients (To, Cc, Bcc)
        recipients = email_message.get_all('To', [])
        cc = email_message.get_all('Cc', [])
        bcc = email_message.get_all('Bcc', [])
        
         # Extract email size
        email_size = len(raw_email) / 1024  # Size in kilobytes
        
        # Get other metadata fields
        flags = email_message.get('flags', '')
        importance = email_message.get('importance', '')
        categories = email_message.get('categories', '')
        body = ''
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == 'text/plain':
                    body = part.get_payload(decode=True)
                    break
        
        # Create the additional_data dictionary
        additional_data = {
            'recipients': recipients,
            'cc': cc,
            'bcc': bcc,
            'flags': flags,
            'importance': importance,
            'categories': categories,
            'body': body,
            'msg_id' : msg_id,
            'thread_id' : thread_id,
            'labels' : labels,
            'email_time' : email_time,
            'email_size' : email_size
        }
        
    except Exception as e:
        print(f"get_email_metadata > Error : {e}")
        
    return additional_data


Driver Code

In [ ]:
try:

    # Get user credentials
    email_address, password = getEmailCredentials()
    
    print("Got the email Credentials")

    # Get IMAP Connection
    with get_imap_server() as imap_server:
        
        print("Got imap server object")

        # Login to the email account
        imap_server.login(email_address, password)
        
        print("Login successful")

        # Get context data
        config = getDatabaseConfig()
        host = config.get('DATABASE','HOST')
        port = config.get('DATABASE','PORT')
        db_username = config.get('DATABASE','USERNAME')
        db_password = config.get('DATABASE','PASSWORD')
        database = config.get('DATABASE','DATABASE')
        
        print("Got database connection data")

        # Create a SQLAlchemy engine and session
        engine = create_engine(f'postgresql://{db_username}:{db_password}@{host}/{database}')
        Session = sessionmaker(bind=engine)
        
        print("Database session engine created.")

        # Connect to database
        with Session() as session:
            
            print("Database session created")
            
            # Add email address from which you want to receive emails from
            email_from_addresses = ['nandanchitale@gmail.com']
            
            # Iterate over sender's list
            for sender_email in email_from_addresses:
                
                # Get search parameters
                search_filter_string = get_serach_string(email_from=sender_email)
                
                print(f"Serach string : {search_filter_string}")
                
                
                # Select the mailbox/folder you want to fetch emails from (e.g., 'INBOX')
                mailbox = 'INBOX'
                imap_server.select(mailbox)
                
                # Get All emails by search criteria
                status, message_ids = imap_server.search(None, search_filter_string)

                ic(status)

                # If Serach status = OK
                if status == 'OK':
                    
                    print(f"Count of Unseen messages from {sender_email} is : {len(message_ids[0].split())}")
                    
                    # Iterate over emails by reverse order (latest emails first)
                    for message_id in reversed(message_ids[0].split()):
                        
                        # Get id of email ( Each mail have unique id )
                        message_id = message_id.decode()
                        
                        print(f"Getting data for {message_id}")
                        
                        # Get Email data from email id and IMAP server Object
                        raw_email, email_message = get_email_data(
                            imap_server=imap_server,
                            message_id=message_id
                        )
                        
                        # _, data = imap_server.fetch(message_id, '(RFC822)')
                        # raw_email = data[0][1]
                        
                        # # Parse the raw email data into an email object
                        # email_message = email.message_from_bytes(raw_email)
                        # ic(email_message)
                        
                        # Extract required email metadata and attachments
                        email_data = extract_email_metadata(email_message)
                        email_metadata = get_email_metadata(message_id, imap_server)
                        attachment_data = extract_attachments(email_message)
                        
                        ic(email_data)
                        ic(email_metadata)
                        
                        # for attachment in attachment_data:
                        #     saveEmailAttachment(attachment['part'])
                        
                        # Save email data to the database
                        save_email_data(session, email_data, attachment_data, email_metadata)
        
        # Logout from IMAP Server
        imap_server.logout()
    
except Exception as e:
    print(f"Error in driver code : {e}")